In [3]:
#!git clone https://github.com/William-Chittavong/torchscale.git

In [4]:
# LEFT OFF: hook has wrong path for imports. change them

In [5]:
!pip uninstall torchscale -y

Found existing installation: torchscale 0.2.0
Uninstalling torchscale-0.2.0:
  Successfully uninstalled torchscale-0.2.0


In [6]:
!pip install git+https://github.com/William-Chittavong/torchscale.git

  Cloning https://github.com/William-Chittavong/torchscale.git to /tmp/pip-req-build-hwpp4owu
  Running command git clone --filter=blob:none --quiet https://github.com/William-Chittavong/torchscale.git /tmp/pip-req-build-hwpp4owu
  Resolved https://github.com/William-Chittavong/torchscale.git to commit c199aff41ceaba9a8fb649fabc9f58150ba3a977
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for torchscale, since package 'wheel' is not installed.
  Running setup.py install for torchscale ... done


In [7]:
import math
import torch
import torch.nn as nn

from torchscale.model.BEiT3 import create_beit3_retrieval_model

In [8]:
from torchscale.component.hook import HookManager
hook = HookManager()
model_size = "base"
img_size = 224

retrieve_model = create_beit3_retrieval_model(model_size='base',hook_manager= hook, img_size=224)

In [9]:
device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

In [10]:
from torchvision import transforms

from torchscale.component.beit3_utils import load_model_and_may_interpolate


from transformers import XLMRobertaTokenizer
from PIL import Image

tokenizer = XLMRobertaTokenizer("/home/william/Documents/GitHub/torchscale/beit3.spm")

image_path = "/home/william/project/images/catdog.png"
image = Image.open(image_path)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the size expected by the model
    transforms.ToTensor(),          # Convert to tensor
])

img_tensor = transform(image).unsqueeze(0).requires_grad_(True)
img_tensor = img_tensor.to(device)

checkpoint_path = "https://github.com/addf400/files/releases/download/beit3/beit3_base_patch16_224.pth"

image_text_contrastive_checkpoint = "https://github.com/addf400/files/releases/download/beit3/beit3_base_itc_patch16_224.pth"


# Load the checkpoint into vqa_fixed
#load_model_and_may_interpolate(checkpoint_path, vqa_model, model_key='model', model_prefix='')

load_model_and_may_interpolate(image_text_contrastive_checkpoint, retrieve_model, model_key='model', model_prefix='')




Load ckpt from https://github.com/addf400/files/releases/download/beit3/beit3_base_itc_patch16_224.pth
Load state_dict by model_key = model


In [11]:
from torchscale.component.prs_hook import hook_prs_logger
retrieve_model.to(device)
retrieve_model.eval()

encoder_embed_dim = 768
prs = hook_prs_logger(retrieve_model,encoder_embed_dim  ,device)


In [12]:
retrieve_model

BEiT3ForRetrieval(
  (beit3): BEiT3(
    (text_embed): TextEmbedding(64010, 768)
    (vision_embed): VisionEmbedding(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): Encoder(
      (dropout_module): Dropout(p=0.0, inplace=False)
      (embed_positions): MutliwayEmbedding(
        (A): PositionalEmbedding(199, 768)
        (B): PositionalEmbedding(1024, 768)
      )
      (layers): ModuleList(
        (0-11): 12 x EncoderLayer(
          (self_attn): MultiheadAttention(
            (k_proj): MultiwayNetwork(
              (A): Linear(in_features=768, out_features=768, bias=True)
              (B): Linear(in_features=768, out_features=768, bias=True)
            )
            (v_proj): MultiwayNetwork(
              (A): Linear(in_features=768, out_features=768, bias=True)
              (B): Linear(in_features=768, out_features=768, bias=True)
            )
            (q_proj): MultiwayNetwork(
              (A): Linear(in_features=768, out_featu

In [13]:
with torch.no_grad():
    rep = retrieve_model(image=img_tensor,only_infer = True)
    attentions,mlps = prs.finalize()


RuntimeError: stack expects a non-empty TensorList